In [18]:
import math
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [13]:
data_path = '../../data/clasdb_pi_plus_n.txt'

In [33]:
df = pd.read_csv(data_path, delimiter='\t', header=None)
df.columns = ['Ebeam', 'W', 'Q2', 'cos_theta', 'phi', 'dsigma_dOmega', 'error', 'id']
df.loc[8314:65671, 'Ebeam'] = 5.754  # peculiarity of this dataset.
df = df[~((df.Ebeam == 5.754) & (~df.Q2.isin([1.715, 2.050, 2.445, 2.915, 3.480, 4.155])))] # peculiarity of this dataset #2
df['phi'] = df.phi.apply(lambda x: math.radians(x))
df['weight'] = df['error'].apply(lambda x: x and 1 / x or 100)  # x and 1 / x or 100  is just a reversed error but with validation 1/0 error in this case it will return 100
df = df.drop('id', axis=1)
df = df.drop_duplicates(subset=['Ebeam', 'W', 'Q2', 'cos_theta', 'phi'])
df['cos_theta_rounded'] = df['cos_theta'].apply(lambda x: round(x,2))

In [34]:
df_a = pd.read_csv('../../data/final_table.csv', header=None, sep='\t',
                    names=['Channel', 'MID', 'Wmin', 'Wmax', 'Q2min', 'Q2max', 'Cos(theta)', 'sigma_t', 'd_sigma_t',
                    'sigma_l', 'd_sigma_l', 'sigma_tt', 'd_sigma_tt', 'sigma_lt', 'd_sigma_lt', 'eps'])
df_a = df_a[df_a.Channel.isin([8, 14, 41, 141])]
df_a['W'] = (df_a['Wmax'] + df_a['Wmin'])/2
df_a['Q2'] = (df_a['Q2max'] + df_a['Q2min'])/2
df_a['cos_theta'] = df_a['Cos(theta)']
df_a['cos_theta_rounded'] = df_a['cos_theta'].apply(lambda x: round(x,2))

In [52]:
df_data = df.merge(df_a, how='left', on=['W', 'Q2', 'cos_theta_rounded'])

df_data['A'] = df_data.sigma_t + df_data.sigma_l
df_data['A_error'] = df_data.d_sigma_t + df_data.d_sigma_l

df_data['B'] = df_data.sigma_tt
df_data['B_error'] = df_data.d_sigma_tt

df_data['C'] = df_data.sigma_lt
df_data['C_error'] = df_data.d_sigma_lt


df_data['cos_theta'] = df_data['cos_theta_x']
df_data = df_data[['Ebeam', 'W', 'Q2', 'cos_theta', 'phi',
                    'dsigma_dOmega', 'error', 'A', 'A_error',
                    'B', 'B_error', 'C', 'C_error', 'eps']]

In [55]:
df_data.to_csv('../../data/df_with_ABC.csv')

In [56]:
df_data

,Ebeam,W,Q2,cos_theta,phi,dsigma_dOmega,error,A,A_error,B,B_error,C,C_error,eps
0,1.515,1.11,0.3,0.991445,0.261799,15.3700,5.264366,4.414180,0.617435,-0.063572,0.730597,-0.856855,0.397176,0.889562
1,1.515,1.11,0.3,0.991445,0.785398,4.5110,1.743136,4.414180,0.617435,-0.063572,0.730597,-0.856855,0.397176,0.889562
2,1.515,1.11,0.3,0.991445,1.308997,4.4780,1.611260,4.414180,0.617435,-0.063572,0.730597,-0.856855,0.397176,0.889562
3,1.515,1.11,0.3,0.991445,1.832596,5.1360,1.523529,4.414180,0.617435,-0.063572,0.730597,-0.856855,0.397176,0.889562
4,1.515,1.11,0.3,0.991445,2.356194,5.0780,1.219442,4.414180,0.617435,-0.063572,0.730597,-0.856855,0.397176,0.889562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82130,5.499,2.01,4.0,0.975000,3.730641,0.1012,0.043165,0.205037,0.110653,-0.069930,0.130671,0.010919,0.049560,0.566140
82131,5.499,2.01,4.0,0.975000,3.992441,0.1199,0.076638,0.205037,0.110653,-0.069930,0.130671,0.010919,0.049560,0.566140
82132,5.499,2.01,4.0,0.975000,4.646939,0.1578,0.095391,0.205037,0.110653,-0.069930,0.130671,0.010919,0.049560,0.566140
82133,5.499,2.01,4.0,0.975000,4.777839,0.2346,0.158557,0.205037,0.110653,-0.069930,0.130671,0.010919,0.049560,0.566140
